In [6]:
pip install opencv-python mediapipe

In [17]:
import cv2
import mediapipe as mp
import random
import math
import os
from collections import deque

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Utility to calculate angle between three landmarks
def calculate_angle(a, b, c):
    ab = (a[0] - b[0], a[1] - b[1])
    cb = (c[0] - b[0], c[1] - b[1])
    dot = ab[0] * cb[0] + ab[1] * cb[1]
    ab_len = math.hypot(*ab)
    cb_len = math.hypot(*cb)
    if ab_len * cb_len == 0:
        return 0
    angle = math.acos(dot / (ab_len * cb_len))
    return math.degrees(angle)

# Improved gesture logic using angles
def get_hand_gesture(landmarks):
    coords = [(lm.x, lm.y) for lm in landmarks]

    # Angles for fingers: MCP-PIP-TIP
    finger_angles = []
    for base, mid, tip in [(5, 6, 8), (9, 10, 12), (13, 14, 16), (17, 18, 20)]:
        angle = calculate_angle(coords[base], coords[mid], coords[tip])
        finger_angles.append(angle)

    # Thumb angle: CMC-MCP-IP
    thumb_angle = calculate_angle(coords[1], coords[2], coords[4])

    # Thresholds:
    finger_states = [angle < 60 for angle in finger_angles]  # True = curled
    thumb_curled = thumb_angle < 50

    # Classify
    if all(finger_states) and thumb_curled:
        return 'Rock'
    elif not any(finger_states) and not thumb_curled:
        return 'Paper'
    elif finger_states == [False, False, True, True] and thumb_curled:
        return 'Scissors'
    return 'Unknown'

# Determine winner
def get_winner(player, ai):
    if player == ai:
        return 'Draw'
    wins = {'Rock': 'Scissors', 'Paper': 'Rock', 'Scissors': 'Paper'}
    return 'Player' if wins[player] == ai else 'AI'

# Draw helper
def draw_text(img, text, pos, scale=1, color=(0,255,0)):
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, scale, color, 2)

# Main game
def play_rps_from_video(input_path, output_path='yolo.mp4', rounds_to_win=3):
    if not os.path.exists(input_path):
        print(f"File not found: {input_path}")
        return

    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                           min_detection_confidence=0.3, min_tracking_confidence=0.3)

    buffer = deque(maxlen=int(fps * 0.8))  # 0.8s buffer
    scores = {'Player': 0, 'AI': 0}
    state = 'Countdown'
    gesture_locked = None
    ai_move = 'None'
    result_text = ''
    countdown_frames = int(fps * 1.5)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        gesture = 'None'
        if results.multi_hand_landmarks:
            for lm in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, lm, mp_hands.HAND_CONNECTIONS)
                gesture = get_hand_gesture(lm.landmark)

        # Only consider valid gestures
        if gesture in ['Rock', 'Paper', 'Scissors']:
            buffer.append(gesture)

        # Game logic
        if state == 'Countdown':
            draw_text(frame, f"Hold your gesture!", (30, 60), 1, (0, 255, 255))
            draw_text(frame, f"Countdown: {int((countdown_frames - frame_count)/fps)}", (30, 100), 1, (0, 255, 255))
            if frame_count >= countdown_frames:
                if len(buffer) > 0 and all(g == buffer[0] for g in buffer):
                    gesture_locked = buffer[0]
                    ai_move = random.choice(['Rock', 'Paper', 'Scissors'])
                    winner = get_winner(gesture_locked, ai_move)
                    if winner != 'Draw':
                        scores[winner] += 1
                    result_text = f"You: {gesture_locked}, AI: {ai_move} => {winner} wins"
                    state = 'ShowResult'
                    frame_count = 0
                else:
                    buffer.clear()
                    frame_count = 0
            else:
                frame_count += 1

        elif state == 'ShowResult':
            draw_text(frame, result_text, (30, 60), 0.9, (0, 255, 0))
            draw_text(frame, f"Score - You: {scores['Player']} | AI: {scores['AI']}", (30, 100), 0.8)
            frame_count += 1
            if frame_count > fps * 2:
                if scores['Player'] == rounds_to_win or scores['AI'] == rounds_to_win:
                    state = 'GameOver'
                else:
                    state = 'Countdown'
                    buffer.clear()
                    frame_count = 0

        elif state == 'GameOver':
            winner = 'You' if scores['Player'] == rounds_to_win else 'AI'
            draw_text(frame, f"Game Over - {winner} Wins!", (30, height // 2), 1.2, (0, 0, 255))

        # Always show current guess
        if state != 'GameOver':
            draw_text(frame, f"Detected: {gesture}", (30, height - 40), 0.8, (255, 255, 255))

        out.write(frame)

    cap.release()
    out.release()
    hands.close()
    print(f"✅ Game finished. Output saved to: {output_path}")

# Entry point
if __name__ == "__main__":
    input_video_path = "/content/hi.mp4"
    play_rps_from_video(input_video_path)


✅ Game finished. Output saved to: yolo.mp4


In [18]:
# output video: https://drive.google.com/file/d/19pXnw02EsM8FXA-8bSpdk60FrNhs5Uw6/view?usp=drivesdk